# Setup

Piz Daint uses an outdated package selection, so in this notebook we'll prepare a custom depot with up-to-date packages. Let's start by inspecting the current state:

In [1]:
using Pkg
if VERSION == v"1.6.1"
    Pkg.DEFAULT_IO[] = stdout # Pkg.jl#2542
end

In [2]:
Pkg.activate(@__DIR__)
Pkg.status()

  Activating environment at `~/cscs_gpu_course/Project.toml`
      Status `~/cscs_gpu_course/Project.toml`
  [79e6a3ab] Adapt v3.3.1
→ [6e4b80f9] BenchmarkTools v1.2.0
→ [052768ef] CUDA v3.5.0 `https://github.com/JuliaGPU/CUDA.jl.git#4a575ea`
→ [72cfdca4] CUDAKernels v0.3.1
  [3da002f7] ColorTypes v0.11.0
  [53c48c17] FixedPointNumbers v0.8.4
  [61eb1bfa] GPUCompiler v0.13.7
  [7073ff75] IJulia v1.23.2
→ [916415d5] Images v0.24.1
→ [63c18a36] KernelAbstractions v0.7.1
→ [5e47fb64] TestImages v1.6.2
→ [bc48ee85] Tullio v0.3.2
        Info packages marked with → not downloaded, use `instantiate` to download


If you haven't run this notebook, many packages will be missing. We can't simply instantiate because the homedir filesystem is really slow, doesn't support `git`, and sadly hosts the main depot:

In [3]:
DEPOT_PATH

6-element Vector{String}:
 "/users/class99/.julia/1.6.1/daint-gpu"
 "/apps/daint/UES/jenkins/7.0.UP02-20.11/gpu/easybuild/software/jupyterlab/1.2.21-CrayGNU-20.11-batchspawner-cuda/share/IJulia"
 "/apps/daint/UES/jenkins/7.0.UP02-20.11/gpu/easybuild/software/JuliaExtensions/1.6.1-CrayGNU-20.11-cuda/extensions"
 "/apps/daint/UES/jenkins/7.0.UP02-20.11/gpu/easybuild/software/Julia/1.6.1-CrayGNU-20.11-cuda/extensions"
 "/apps/daint/UES/jenkins/7.0.UP02-20.11/gpu/easybuild/software/Julia/1.6.1-CrayGNU-20.11-cuda/local/share/julia"
 "/apps/daint/UES/jenkins/7.0.UP02-20.11/gpu/easybuild/software/Julia/1.6.1-CrayGNU-20.11-cuda/share/julia"

We can't simply replace the depot now, because Jupyter has already preloaded quite some packages, so let's write a startup file that replaces our depot before any of that happens:

In [4]:
@show config_dir = joinpath(DEPOT_PATH[1], "config")
isdir(config_dir) || mkdir(config_dir)

@show config_file = joinpath(config_dir, "startup.jl")
open(config_file, "w") do io
    println(io, """
        DEPOT_PATH[1] = "/scratch/snx3000/$(ENV["USER"])/julia"
    """)
end

config_dir = joinpath(DEPOT_PATH[1], "config") = "/users/class99/.julia/1.6.1/daint-gpu/config"
config_file = joinpath(config_dir, "startup.jl") = "/users/class99/.julia/1.6.1/daint-gpu/config/startup.jl"


For now, since we won't be actually doing anything useful in *this* notebook, we *can* replace the depot just to install the packages we need:

In [5]:
DEPOT_PATH[1] = "/scratch/snx3000/$(ENV["USER"])/julia"

"/scratch/snx3000/class99/julia"

With that in place, we can install packages and verify we have everything we need:

In [6]:
Pkg.instantiate()
Pkg.status()

     Cloning git-repo `https://github.com/JuliaGPU/CUDA.jl.git`
   Installed ProgressMeter ───────────── v1.7.1
   Installed ImageAxes ───────────────── v0.6.9
   Installed FFTW_jll ────────────────── v3.3.10+0
   Installed ImageMagick ─────────────── v1.2.1
   Installed IntervalSets ────────────── v0.5.3
   Installed OpenEXR ─────────────────── v0.3.2
   Installed Parameters ──────────────── v0.12.3
   Installed StackViews ──────────────── v0.1.1
   Installed FFTViews ────────────────── v0.3.2
   Installed Imath_jll ───────────────── v3.1.2+0
   Installed TestImages ──────────────── v1.6.2
   Installed ImageDistances ──────────── v0.2.13
   Installed ColorVectorSpace ────────── v0.9.7
   Installed Distances ───────────────── v0.10.6
   Installed Parsers ─────────────────── v2.1.2
   Installed PaddedViews ─────────────── v0.5.10
   Installed ImageMagick_jll ─────────── v6.9.12+0
   Installed AxisAlgorithms ──────────── v1.0.1
   Installed AxisArrays ──────────────── v0.4.4
   Installed

Be sure to restart all kernels after this, so that the new environment will be used!

In addition, shut down this (and any other) kernel before launching a new one, because the GPUs on Piz Daint run in exclusive mode (i.e. they only allow a single active compute process).